# SQLAlchemy untuk Database

Pada bagian ini akan diperlihatkan penggunaan SQLAlchemy-Flask untuk mempermudah pembuatan dan mengakses database. SQLAlchemy yang digunakan merupakan sebuah ekstensi untuk flask, bukan standalone.

In [9]:
from flask import Flask, jsonify, abort, make_response, request
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

## Database Configuration

Disini menggunakan mysql sehingga pada db_uri berupa `mysql+mysqldb` dimana `mysqldb` merupakan driver python yang digunakan untuk mengakses mysql.

In [10]:
user = 'root'
password = 'root'
host = 'localhost'
port = 3306
dbname = 'coba'

db_uri = 'mysql+mysqldb://%s:%s@%s:%d/%s' % (user, password, host, port, dbname)

In [11]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = db_uri
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True
db = SQLAlchemy(app)

## Membuat Tabel
> Misal kita ingin membuat dua database `anggota` dan `kegiatan` dengan hubungan sebagai berikut. 

![](image/db1.png)

> Untuk menghubungkan antara kegiatan dan peserta, maka dibentuk tabel `many-to-many` dengan nama `daftar_hadir`.

> Untuk tipe data pada masing masing kolom, dapat dilihat referensi di http://flask-sqlalchemy.pocoo.org/2.3/models/ 

Tipe data | Penjelasan
-- | --
Integer	| an integer
String(size) | a string with a maximum length (optional in some databases, e.g. PostgreSQL)
Text | some longer unicode text
DateTime | date and time expressed as Python datetime object.
Float | stores floating point values
Boolean | stores a boolean value
PickleType | stores a pickled Python object
LargeBinary | stores large arbitrary binary data

In [12]:
class Anggota(db.Model):
    __table_args__ = {'extend_existing': True}  
    
    id = db.Column(db.Integer, primary_key=True)
    nama = db.Column(db.String(80), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    telegram = db.Column(db.String(64), unique=True, nullable=False)
    tanggal_terdaftar = db.Column(db.DateTime, default=datetime.utcnow)
    ikut_serta = db.relationship('Kegiatan', secondary='daftar_hadir', backref=db.backref('peserta', lazy='dynamic'))

    def __repr__(self):
        return '<Anggota %r>' % self.telegram

In [13]:
class Kegiatan(db.Model):
    __table_args__ = {'extend_existing': True} 
    
    id = db.Column(db.Integer, primary_key=True)
    tema =  db.Column(db.Text, nullable=False)
    pembicara = db.Column(db.Text,  nullable=False)
    tempat = db.Column(db.Text,  nullable=False)
    waktu = db.Column(db.DateTime, nullable=False)

    def __repr__(self):
        return '<Kegiatan %r>' % self.tema

> Selanjutnya untuk membuat many-to-many ada sedikit perbedaan untuk membuatnya

In [14]:
daftar_hadir = db.Table('daftar_hadir',
    db.Column('kegiatan',db.Integer, db.ForeignKey('kegiatan.id'), index=True),
    db.Column('peserta',db.Integer, db.ForeignKey('anggota.id'), index=True),
)

> Ketika semua pengaturan sudah dibuat, selanjutnya akan dibuat tabel berdasarkan kelas yang telah dibuat dengan perintah `db.create_all()`.

In [15]:
db.create_all()

> Jika dilihat dari phpmyadmin akan terbuat tabel sebagai berikut

![](image/db2.png)

**Tabel anggota**
![](image/db3.png)

**Tabel kegiatan**
![](image/db4.png)

**Tabel daftar_hadir**
![](image/db5.png)

> Yang perlu diperhatikan adalah pada `Class Anggota` terdapat variabel yang tidak ada di tabel anggota yaitu:

```python3
ikut_serta = db.relationship('Kegiatan', secondary='daftar_hadir', backref=db.backref('peserta', lazy='dynamic'))
```

> ketika menggunakan `db.relationship`, variabel tersebut digunakan untuk mengakses data dari tabel lainnya, dalam hal ini `Kegiatan` dan `daftar_hadir`. Untuk lebih jelasnya akan diperlihatkan pada contoh dibawah. Namun sebelumnya kita akan memasukan beberapa data terlebih dahulu kedalam tabel.

## Memasukan Data Pada Tabel

> Pertama kita akan membuat objek dari `Class Anggota`. Anggota mensyaratkan tiga data yang harus diisi yaitu `nama`, `email`, dan `telegram`. Sedangkan `id` akan di autoincrement dan `tanggal_terdaftar` memiliki nilai default `datetime.utcnow`.

In [16]:
nama = 'tegar'
email = 'egar@imansyah.com'
telegram = 'tegarimansyah'

user1 = Anggota(nama=nama, email=email, telegram=telegram)

> Lalu kita lihat apa saja isi dari user1

In [22]:
print('nama : ', user1.nama)
print('email : ', user1.email)
print('telegram : ', user1.telegram)
print('id : ', user1.id)
print('tanggal_terdaftar : ', user1.tanggal_terdaftar)

nama :  tegar
email :  egar@imansyah.com
telegram :  tegarimansyah
id :  None
tanggal_terdaftar :  None


> Selanjutnya kita masukan `user1` kedalam database

In [23]:
db.session.add(user1)
db.session.commit()

> Data berhasil ditambahkan

![](image/db6.png)

> Selanjutnya kita lihat lagi object user1. Dia akan mendapatkan nilai untuk `id` dan `tanggal_terdaftar`.

In [24]:
print('nama : ', user1.nama)
print('email : ', user1.email)
print('telegram : ', user1.telegram)
print('id : ', user1.id)
print('tanggal_terdaftar : ', user1.tanggal_terdaftar)

nama :  tegar
email :  egar@imansyah.com
telegram :  tegarimansyah
id :  1
tanggal_terdaftar :  2018-04-22 02:29:54


> Kita akan membuat data anggota yang lebih banyak

In [26]:
user2 = Anggota(nama='udin', email='udin@gmail.com', telegram='udin')
user3 = Anggota(nama='sari', email='sari@gmail.com', telegram='sari')
user4 = Anggota(nama='agus', email='agus@gmail.com', telegram='agus')
user5 = Anggota(nama='budi', email='budi@gmail.com', telegram='budi')

for user in [user2, user3, user4, user5]:
    db.session.add(user)

db.session.commit()

> Hal yang sama juga dilakukan untuk kegiatan. Tapi sebelumnya kita harus melakukan format untuk tipe data waktu dengan membuat sebuah fungsi `waktu`

In [29]:
def waktu(data):
    return datetime.strptime(data, '%d %b %Y %H:%M') 

waktu('18 Apr 2018 18:00')

datetime.datetime(2018, 4, 18, 18, 0)

In [30]:
kegiatan1 = Kegiatan(tema = 'Belajar Python Basic', pembicara = 'Surabaya.py', 
                     tempat = 'Cafe A', waktu = waktu('20 Apr 2018 18:00'))

kegiatan2 = Kegiatan(tema = 'Belajar Python Advanced', pembicara = 'Guido Van Rossum', 
                     tempat = 'Cafe B', waktu = waktu('30 Apr 2018 18:00'))

In [31]:
for kegiatan in [kegiatan1, kegiatan2]:
    db.session.add(kegiatan)

db.session.commit()

**Data Anggota Terakhir**
![](image/db7.png)

**Data Kegiatan Terakhir**
![](image/db8.png)

## Menggabungkan Data

> Selanjutnya adalah menggabungkan data anggota dan kegiatan. Misal daftar hadir sebagai berikut

Id Kegiatan | Tema Kegiatan | Id Anggota | Nama Anggota
--|--|--|--
1 | Belajar Python Basic | 1 | tegar
1 | Belajar Python Basic | 2 | udin
1 | Belajar Python Basic | 3 | sari
1 | Belajar Python Basic | 4 | agus
1 | Belajar Python Basic | 5 | budi
--|--|-- | --
2 | Belajar Python Advanced | 1 | tegar
2 | Belajar Python Advanced | 3 | sari
2 | Belajar Python Advanced | 4 | agus

In [33]:
kegiatan1.peserta.append(user1)
kegiatan1.peserta.append(user2)
kegiatan1.peserta.append(user3)
kegiatan1.peserta.append(user4)
kegiatan1.peserta.append(user5)

kegiatan2.peserta.append(user1)
kegiatan2.peserta.append(user3)
kegiatan2.peserta.append(user4)

db.session.commit()

> Informasi tersebut akan disimpan pada tabel many-to-many `daftar_hadir`.

![](image/db9.png)

> Mungkin akan bingung padahal `Class Kegiatan` tidak memiliki variabel `peserta`, namun sebenarnya sudah diinisiasi pada `Class Anggota` pada parameter `backref`:

ikut_serta = db.relationship('Kegiatan', secondary='daftar_hadir', backref=db.backref(**'peserta'**, lazy='dynamic'))

> Sebenarnya bisa juga dibalik pada sisi anggota misalnya **user2 memutuskan untuk ikut kegiatan 2**

In [34]:
user2.ikut_serta.append(kegiatan2)
db.session.commit()

> Lalu sekarang bagian yang menariknya yaitu kita dapat **melihat semua kegiatan dari seorang anggota** atau **melihat semua peserta dari suatu kegiatan**

In [39]:
print('Nama peserta kegiatan %s:' % (kegiatan1.tema))
for peserta in kegiatan1.peserta:
    print('* ', peserta.nama)

print('\n')    

print('Email peserta kegiatan %s:' % (kegiatan2.tema))
for peserta in kegiatan2.peserta:
    print('* ', peserta.email)

print('\n')    

print('Daftar kegiatan yang diikuti %s:' % (user1.nama))
for kegiatan in user1.ikut_serta:
    print('* ', kegiatan.tema)

Nama peserta kegiatan Belajar Python Basic:
*  tegar
*  udin
*  sari
*  agus
*  budi


Email peserta kegiatan Belajar Python Advanced:
*  egar@imansyah.com
*  sari@gmail.com
*  agus@gmail.com
*  udin@gmail.com


Daftar kegiatan yang diikuti tegar:
*  Belajar Python Basic
*  Belajar Python Advanced


## Query

> Beberapa macam query dapat dilihat pada http://flask-sqlalchemy.pocoo.org/2.3/queries/

In [41]:
# Mendapat semua anggota dari database

semua_anggota = Anggota.query.all()
semua_anggota

[<Anggota 'tegarimansyah'>,
 <Anggota 'udin'>,
 <Anggota 'sari'>,
 <Anggota 'agus'>,
 <Anggota 'budi'>]

In [42]:
# Melihat semua email dari query sebelumnya

for anggota in semua_anggota:
    print(anggota.email)

egar@imansyah.com
udin@gmail.com
sari@gmail.com
agus@gmail.com
budi@gmail.com


In [45]:
# Mencari query tertentu

udin = Anggota.query.filter_by(telegram='udin').first()
udin

<Anggota 'udin'>

In [46]:
# Melihat kegiatan yang diikuti oleh anggota

udin.ikut_serta

[<Kegiatan 'Belajar Python Basic'>, <Kegiatan 'Belajar Python Advanced'>]

In [47]:
# Melihat waktu dari semua kegiatan yang diikuti udin
for kegiatan in udin.ikut_serta:
    print(kegiatan.waktu)

2018-04-20 18:00:00
2018-04-30 18:00:00


## Update data

In [48]:
print('nama : ', user1.nama)
print('email : ', user1.email)
print('telegram : ', user1.telegram)
print('id : ', user1.id)
print('tanggal_terdaftar : ', user1.tanggal_terdaftar)

nama :  tegar
email :  egar@imansyah.com
telegram :  tegarimansyah
id :  1
tanggal_terdaftar :  2018-04-22 02:29:54


In [50]:
user1.email = 'tegar@gmail.com'
db.session.commit()

In [51]:
print('nama : ', user1.nama)
print('email : ', user1.email)
print('telegram : ', user1.telegram)
print('id : ', user1.id)
print('tanggal_terdaftar : ', user1.tanggal_terdaftar)

nama :  tegar
email :  tegar@gmail.com
telegram :  tegarimansyah
id :  1
tanggal_terdaftar :  2018-04-22 02:29:54


## Delete Data

> Sebelumnya kita telah memiliki objek udin dan kita akan menghapus data dari objek udin pada database.

In [52]:
udin

<Anggota 'udin'>

In [53]:
db.session.delete(udin)
db.session.commit()

> Dapat dilihat dari anggota komunitas, sudah tidak ada udin

In [54]:
semua_anggota = Anggota.query.all()
semua_anggota

[<Anggota 'tegarimansyah'>,
 <Anggota 'sari'>,
 <Anggota 'agus'>,
 <Anggota 'budi'>]

> Hal tersebut juga mempengaruhi tabel lain

In [56]:
for peserta in kegiatan1.peserta:
    print(peserta.nama)

tegar
sari
agus
budi
